In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# autogluon 라이브러리를 써보자!!

In [ ]:
pip install autogluon

In [ ]:
## tabular(테이블) 형식의 데이터를 다루는 모듈을 다운로드한다.
from autogluon.tabular import TabularDataset, TabularPredictor

## 분석의 절차

### A. 데이터 입력

> 문제를 받아오는 과정으로 비유할 수 있다.

In [ ]:
tr = TabularDataset('/kaggle/input/titanic/train.csv')  ## 학습할 데이터
tst = TabularDataset('/kaggle/input/titanic/test.csv')

## tr = pd.read_csv('/kaggle/input/titanic/train.csv')
## tst

### B. Predictor 생성
> 문제를 풀 학생을 생성하는 과정으로 비유할 수 있다.

In [ ]:
predictr = TabularPredictor('Survived') ## target variable이 들어있는 데이터프레임, 변수 철자는 임의로 틀리게 설정

> target variable이 들어있는 데이터셋을 추출한다.

In [ ]:
type(predictr)

### C. 적합(fit)
> 학습 과정에 해당

In [ ]:
predictr.fit(tr) ## 학생(predictr)에게 문제(tr)를 주어 학습을 시킴(predictr.fit(tr))

학습 완료, 이에 따라 리더보드를 확인한다. (모의고사 채점)

In [ ]:
predictr.leaderboard()

**score_val이 의미하는 것**
* 실제로 predictr가 학습한 것은?
    > predictor와 train set이 있고, train set에 데이터가 1000개 있다고 하면 해당 데이터를 전부 가용하지 않는다.
    > * 800개를 사용한다고 하면 200개는 학습하지 않고 답을 맞춰 보는 식이다.
    > 
    > * **200개는 왜 남겨두지?**
    > 
    > 문제에서 답을 찾는 규칙이 맞는지, 다른 데이터들에 대해서도 일반화시킬 수 있는 지 테스트 해보면 좋을 것 같다. 따라서 나머지 데이터셋에서 분석을 해본다.
    >
    > **실제 테스트에서 잘하기 위한 자체적 테스트셋에 해당**, 200개의 나머지 테스트용 데이터셋을 validation set이라 일컫는다.


||train|val|
|-|-|-|
|학생1|95%|72%|
|학생2|80%|80%|
|...|...|...|

train(연습문제)만 계속 푼 것 보다, val(모의고사)에서 가장 높은 점수를 받은 것이 유의미할 것.

> 가장 높은 점수를 받은 ```WeightedEnsemble_L2```모델을 사용해보자

### D. 예측(predict)
> 학습 이후에 문제를 푸는 과정으로 비유.

기존에 했던 분석들

1. 무조건 남자는 죽고, 여자는 사는 형식 0.7x / 0.76555
2. RandomForestClassifier를 사용한 형식 0.8x / 0.77511
3. RandomForestClassifier에서 하이퍼파라미터를 조정한 형식 0.8x / 0.76555 (트레인 셋에서의 분석에서는 더 높았는데 실제 결과는 오히려 더 낮았다.)
**4. WeightedEnsemble_L2모델 사용**

train set을 일단 풀어보자(predict)

In [ ]:
type(tr) ## 처음 보는 것으로 저장되는데 데이터프레임에서 쓸 수 있는 모든 기능들을 다 사용할 수 있다.

In [ ]:
tr.head()

In [ ]:
(tr.Survived == predictr.predict(tr)).mean()

> 정확도가 0.9349나 된다. 상당히 기대가 되는 부분

In [ ]:
predictr.predict(tst)

In [ ]:
tst.assign(Survived = predictr.predict(tst)).loc[:, ['PassengerId', 'Survived']]\
.to_csv('autogluon_submission.csv', index = False)

> 제출 결과 정확도는 0.78947로 지금껏 가장 높은 수치가 나왔다.

## 분기

> 다른 방법을 이용


### A. 데이터

In [ ]:
tr = TabularDataset('/kaggle/input/titanic/train.csv')  ## 학습할 데이터
tst = TabularDataset('/kaggle/input/titanic/test.csv')

## tr = pd.read_csv('/kaggle/input/titanic/train.csv')
## tst

```-```피쳐 엔지니어링

In [ ]:
tr.eval('Fsize = SibSp + Parch')
tst.eval('Fsize = SibSp + Parch')

tr.head()

### B. Predictor 생성

In [ ]:
predictr = TabularPredictor("Survived")

### C. 적합(fit)

In [ ]:
predictr.fit(tr.eval('Fsize = SibSp + Parch'))  ## 새로운 데이터셋을 추가하여 학습

```-```리더보드 확인

In [ ]:
predictr.leaderboard()

### D. 예측(predict)

In [ ]:
(tr.Survived == predictr.predict(tr.eval('Fsize = SibSp + Parch'))).mean()

In [ ]:
tst.assign(Survived = predictr.predict(tst.eval('Fsize = SibSp + Parch'))).loc[:,['PassengerId','Survived']]\
.to_csv("autogluon(Fsize)_submission.csv",index=False)

> 제출 결과 : 점수가 오히려 더 낮아졌음

**더 개선해보자**

```-```피처 엔지니어링

In [ ]:
_tr = tr.eval('Fsize = SibSp + Parch').drop(['SibSp','Parch'],axis=1)
_tst = tst.eval('Fsize = SibSp + Parch').drop(['SibSp','Parch'],axis=1)

_tr.head()
## df.drop(columns = [])
## df.drop([], axis = 1) columns라고 지정해주지 않으면 디폴트로 행을 삭제하기 때문에

### B. Predictor 생성

In [ ]:
predictr = TabularPredictor('Survived')

### C. 적합(fit)

In [ ]:
predictr.fit(_tr)

In [ ]:
predictr.leaderboard()

### D. 예측(predict)

In [ ]:
(tr.Survived == predictr.predict(_tr)).mean()

In [ ]:
predictr.predict(_tr)

In [ ]:
tst.assign(Survived = predictr.predict(_tst)).loc[:, ['PassengerId', 'Survived']]\
.to_csv('autogluon(Fsize,Drop)_submission.csv', index = False)

> 지금껏 가장 높은 결과가 나왔다!

* 다중 공선성 문제를 개선한 결과.